<a href="https://colab.research.google.com/github/Murphy-Mary/MyProject/blob/main/game_recommendations_on_steam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import data


souce : https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

!kaggle datasets download -d antonkozyriev/game-recommendations-on-steam
!unzip /content/game-recommendations-on-steam.zip && rm /content/game-recommendations-on-steam.zip

Mounted at /content/gdrive
 86% 97.0M/112M [00:01<00:00, 67.9MB/s]
100% 112M/112M [00:01<00:00, 69.6MB/s] 
Archive:  /content/game-recommendations-on-steam.zip
  inflating: games.csv               
  inflating: games_metadata.json     
  inflating: recommendations.csv     
  inflating: users.csv               


In [3]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://security.ubuntu.com/ubuntu focal-secur

In [4]:
#Get Install the Java Development Kit
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Get Spark lib -- address need to be change according to the current version of Spark: https://downloads.apache.org/spark/
!wget  http://apache.osuosl.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
#Unzip th Spark
!tar xf spark-3.3.2-bin-hadoop3.tgz
!rm spark-3.3.2-bin-hadoop3.tgz
#install findspark for Python
!pip install -q findspark
#Install extra lib(s)
!pip install -q xlrd
!pip install -q kaggle
!pip install -q kora

#import and set environment for spark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"
import kora

import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession


SPARK_URL = "local[*]"

spark = SparkSession.builder.master(SPARK_URL).getOrCreate()

# Check the pyspark version
import pyspark
print(pyspark.__version__)

--2023-02-26 18:10:02--  http://apache.osuosl.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 64.50.236.52, 64.50.233.100, 140.211.166.134, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|64.50.236.52|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299360284 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.2-bin-hadoop3.tgz’

spark-3.3.2-bin-had 100%[===================>] 285.49M  56.7MB/s    in 10s     

2023-02-26 18:10:12 (27.4 MB/s) - ‘spark-3.3.2-bin-hadoop3.tgz’ saved [299360284/299360284]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.2 MB/s eta 0:00:00
3.3.2


In [5]:
!pip install pyspark_dist_explore

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import numpy as np 
import pandas as pd
import pyspark.sql.functions as f
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, HTML, display_html #usefull to display wide tables
from pyspark_dist_explore import Histogram, hist, distplot, pandas_histogram
from pyspark.sql import Row

from pyspark.sql.functions import lit
from pyspark import SparkConf
from pyspark import SparkContext
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

#sc = pyspark.SparkContext()
sqlContext = pyspark.SQLContext(sc)
%matplotlib inline

/content/spark-3.3.2-bin-hadoop3/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## data

In [7]:
game_tags = spark.read.json("/content/games_metadata.json")
game_tags.printSchema()
game_tags.show()

root
 |-- app_id: long (nullable = true)
 |-- description: string (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)

+------+--------------------+--------------------+
|app_id|         description|                tags|
+------+--------------------+--------------------+
| 10090|Call of Duty is b...|[Zombies, World W...|
| 13500|Enter the dark un...|[Action, Adventur...|
| 22364|                    |            [Action]|
|113020|Monaco: What's Yo...|[Co-op, Stealth, ...|
|226560|Escape Dead Islan...|[Zombies, Adventu...|
|249050|Dungeon of the En...|[Roguelike, Strat...|
|250180|“METAL SLUG 3”, t...|[Arcade, Classic,...|
|253980|Experience incred...|[RPG, Action, Fan...|
|261550|A strategy/action...|[Medieval, Strate...|
|271850|                    |[Strategy, Simula...|
|282900|Packed with fast-...|[Anime, JRPG, Fem...|
| 19810|Lead a team of do...|  [Action, Tactical]|
| 15270|In a ferocious Ar...|[Horror, Action, ...|
| 21130|Build th

In [8]:
game_tags.count()

21236

In [9]:
game = spark.read.csv('/content/games.csv',header=True)
# c0ls = ["user_reviews","price_final","price_original","discount"]
game = game.withColumn("user_reviews",game.user_reviews.cast('double'))
game = game.withColumn("price_final",game.price_final.cast('double'))
game = game.withColumn("price_original",game.price_original.cast('double'))
game = game.withColumn("discount",game.discount.cast('double'))
# game = game.withColumn("mac",game.date_release.cast('Boolean'))
# game = game.withColumn("mac",game.date_release.cast('Boolean'))
# game = game.withColumn("win",game.date_release.cast('Boolean'))
# game = game.withColumn("steam_deck",game.date_release.cast('Boolean'))
game.printSchema()
game.show()

root
 |-- app_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- date_release: string (nullable = true)
 |-- win: string (nullable = true)
 |-- mac: string (nullable = true)
 |-- linux: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- positive_ratio: string (nullable = true)
 |-- user_reviews: double (nullable = true)
 |-- price_final: double (nullable = true)
 |-- price_original: double (nullable = true)
 |-- discount: double (nullable = true)
 |-- steam_deck: string (nullable = true)

+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
|app_id|               title|date_release| win|  mac|linux|         rating|positive_ratio|user_reviews|price_final|price_original|discount|steam_deck|
+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
| 10090|Call of 

In [10]:
game.count()

21236

In [11]:
recomm = spark.read.csv('/content/recommendations.csv',header=True)
recomm = recomm.withColumn("date",recomm.date.cast('date'))
recomm = recomm.withColumn("is_recommended",recomm.is_recommended.cast('Boolean'))
recomm = recomm.withColumn("hours",recomm.hours.cast('double'))
recomm.printSchema()
recomm.show()

root
 |-- app_id: string (nullable = true)
 |-- helpful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- date: date (nullable = true)
 |-- is_recommended: boolean (nullable = true)
 |-- hours: double (nullable = true)
 |-- user_id: string (nullable = true)
 |-- review_id: string (nullable = true)

+-------+-------+-----+----------+--------------+-----+-------+---------+
| app_id|helpful|funny|      date|is_recommended|hours|user_id|review_id|
+-------+-------+-----+----------+--------------+-----+-------+---------+
| 975370|      0|    0|2022-12-12|          true| 36.3|  14546|        0|
| 304390|      4|    0|2017-02-17|         false| 11.5|    797|        1|
|1085660|      2|    0|2019-11-17|          true|336.5|  66915|        2|
| 703080|      0|    0|2022-09-23|          true| 27.4|  68251|        3|
| 526870|      0|    0|2021-01-10|          true|  7.9|   6647|        4|
| 306130|      0|    0|2021-10-10|          true|  8.6|  12866|        5|
| 238960|      0

In [ ]:
recomm.orderBy(f.col("date")).show()

In [ ]:
recomm.count()

In [ ]:
user = spark.read.csv('/content/users.csv',header=True)
user.printSchema()
user.show()

In [ ]:
user.count()

# Prepare Section

In [ ]:
game_tng = game.join(game_tags,["app_id"],"full")

In [ ]:
game_tng.show()
# game_tng.select('tags').collect()[0][0][0]
# game_tng.describe().show()

In [ ]:
game_tng = game_tng.filter(game_tng.app_id.isNotNull()\
                           |game_tng.title.isNotNull()\
                           |game_tng.win.isNotNull()\
                           |game_tng.rating.isNotNull()\
                           |game_tng.price_final.isNotNull()\
                           |game_tng.price_original.isNotNull()\
                           |game_tng.description.isNotNull()\
                           |game_tng.tags.isNotNull()\
                           ).sort("app_id") 


In [ ]:
game_tng.count()

In [ ]:
game_tng.show()

-----------------------------------------------------


In [ ]:
user_recc = recomm.join(user,["user_id"],"full")

In [ ]:
user_recc.show()
# user_recc.describe().show()

In [ ]:
user_recc = user_recc.filter(user_recc.app_id.isNotNull()).sort("user_id") 

In [ ]:
user_recc.count()

In [ ]:
user_recc.show()

In [ ]:
all_data = game_tng.join(user_recc,["app_id"],"full")

In [ ]:
all_data.printSchema()

In [ ]:
# all_data.count()
all_data.show()
# app_id,title,win

In [ ]:
#Drop Null
all_data = all_data.filter(all_data.app_id.isNotNull()|all_data.title.isNotNull()|all_data.win.isNotNull()).sort("app_id") 

In [ ]:
all_data.count()

In [ ]:
all_data.show()

In [ ]:
all_data.filter(all_data.date_release.isNull()).show()

In [ ]:
all_data.groupBy("app_id",'title').count().filter("count > 1").sort("app_id").show()

In [ ]:
# all_data.write.parquet("/content/gdrive/MyDrive/sizeproject/allgamerecc.parquet")

# EDA Section

In [ ]:
all_data.sort("price_original").show()

## ---- top had user -----

In [ ]:
count_data = all_data.select("app_id","title").groupBy("title").count()\
            .orderBy(f.col("count").desc())

In [ ]:
count_data.show()

In [ ]:
# count_data.plt.plot.bar(x='count')  

In [ ]:
# count_datas = count_data.orderBy(f.col('count').desc())
# count_datas.show(20)

## ---- top buy games ----




In [ ]:
price = all_data.groupBy("app_id","title").sum()

In [ ]:
all_data.select(f.col("app_id"),f.col("title"),f.col("price_original")).filter("app_id == 252490").count()

In [ ]:
all_data.select(f.col("app_id"),f.col("title"),f.col("price_original")).filter("app_id == 252490").show()

In [ ]:
# from pyspark.sql.functions import *
price_ord = price.orderBy(f.col('sum(price_original)').desc())
price_ord.show(20)

## -----Top 10 popular games of 2022 with positive reviews-----

In [ ]:
game.show()

In [ ]:
game_pos = game.select('app_id','title','date_release','user_reviews','positive_ratio').\
            filter((game["date_release"]>= lit('2022-01-01')) &(game["positive_ratio"]>=90))\
            .sort(f.col('user_reviews').desc(),f.col('positive_ratio').desc())

In [ ]:
game_pos.show(10)

## ----Top 10 popular games of 2022 with mixed or lower reviews----


In [ ]:
game_pos2 = game.select('app_id','title','date_release','user_reviews','positive_ratio').\
            filter((game["date_release"]>= lit('2022-01-01')) &(game["positive_ratio"] <= 70))\
            .sort(f.col('user_reviews').desc(),f.col('positive_ratio').desc())

In [ ]:
game_pos2.show(10)

## ---- Top Genre popular ----

In [ ]:
game_tags.printSchema()
game_tags.show()

In [ ]:
genre = all_data.select(f.col("app_id"),f.col("title"),f.col("price_final"),f.explode(f.col("tags")))

In [ ]:
ganre_2 = genre.orderBy(f.col('col').desc())
ganre_2.show(10)

In [ ]:
# genre.show() all_data.groupBy("title").count().filter("count > 100")
ganre_3 = genre.select('title','col').groupBy("col").count().orderBy(f.col('count').desc())
# .filter(ganre_2["col"] != lit("eSports")).orderBy(f.col('col').desc())
ganre_3.show(10)

### Top Gerne Free game

In [ ]:
ganre_4 = ganre_2.select('title','col').filter("price_final == 0").groupBy("col").count().orderBy(f.col('count').desc())

In [ ]:
ganre_4.show(10)

### Top Gerne Paid game

In [ ]:
ganre_5 = ganre_2.select('title','col').filter("price_final != 0").groupBy("col").count().orderBy(f.col('count').desc())

In [ ]:
ganre_5.show(10)

### Top Gerne eSpots game

In [ ]:
ganre_6 =  ganre_2.select('title').filter("col == 'eSports'").groupBy("title").count().orderBy(f.col('count').desc())

In [ ]:
ganre_6.show(10)

## ----Top Gerane Games ----

In [ ]:
ganre_3.show(10)

### By Action

In [ ]:
top_game1 = ganre_2.select('title').filter("col == 'Action'").groupBy("title").count().orderBy(f.col('count').desc())

In [ ]:
top_game1.show(10)

### By Singleplayer

In [ ]:
top_game2 = ganre_2.select('title').filter("col == 'Singleplayer'").groupBy("title").count().orderBy(f.col('count').desc())

In [ ]:
top_game2.show(10)

### By Multiplayer

In [ ]:
top_game3 = ganre_2.select('title').filter("col == 'Multiplayer'").groupBy("title").count().orderBy(f.col('count').desc())

In [ ]:
top_game3.show(10)

### By Co-op

In [ ]:
top_game4 = ganre_2.select('title').filter("col == 'Co-op'").groupBy("title").count().orderBy(f.col('count').desc())

In [ ]:
top_game4.show(10)

### By Adventure

In [ ]:
top_game5 = ganre_2.select('title').filter("col == 'Adventure'").groupBy("title").count().orderBy(f.col('count').desc())

In [ ]:
top_game5.show(10)

### By First-Person

In [ ]:
top_game6 = ganre_2.select('title').filter("col == 'First-Person'").groupBy("title").count().orderBy(f.col('count').desc())

In [ ]:
top_game6.show(10)

### By Open World

In [ ]:
top_game7 = ganre_2.select('title').filter("col == 'Open World'").groupBy("title").count().orderBy(f.col('count').desc())

In [ ]:
top_game7.show(10)

### By Atmospheric

In [ ]:
top_game8 = ganre_2.select('title').filter("col == 'Atmospheric'").groupBy("title").count().orderBy(f.col('count').desc())

In [ ]:
top_game8.show(10)

### By Simulation

In [ ]:
top_game9 = ganre_2.select('title').filter("col == 'Simulation'").groupBy("title").count().orderBy(f.col('count').desc())

In [ ]:
top_game9.show(10)

### By Sandbox

In [ ]:
top_game10 = ganre_2.select('title').filter("col == 'Sandbox'").groupBy("title").count().orderBy(f.col('count').desc())

In [ ]:
top_game10.show(10)

## ----Trend by time----

In [ ]:
trend = all_data.groupBy("title",'date_release').count().orderBy(f.col('count').desc())

In [ ]:
trend.show(10)

### By < 2000


In [ ]:
trend1 = all_data.filter((f.year('date_release') < lit('2000'))).groupBy("title",'date_release').count().orderBy(f.col('count').desc())

In [ ]:
trend1.show()

### By 2000-2005

In [ ]:
trend2 = all_data.filter((f.year('date_release') > lit('2000')) &(f.year('date_release') <= lit('2005'))).groupBy("title",'date_release').count().orderBy(f.col('count').desc())

In [ ]:
trend2.show(10)

### By 2005- 2010 

In [ ]:
trend3 = all_data.filter((f.year('date_release') > lit('2005')) &(f.year('date_release') <= lit('2010'))).groupBy("title",'date_release').count().orderBy(f.col('count').desc())

In [ ]:
trend3.show(10)

### By 2010 - 2015 

In [ ]:
trend4 = all_data.filter((f.year('date_release') > lit('2010')) &(f.year('date_release') <= lit('2015'))).groupBy("title",'date_release').count().orderBy(f.col('count').desc())

In [ ]:
trend4.show(10)

### By 2015 - 2020

In [ ]:
trend5 = all_data.filter((f.year('date_release') > lit('2015')) &(f.year('date_release') <= lit('2020'))).groupBy("title",'date_release').count().orderBy(f.col('count').desc())

In [ ]:
trend5.show(10)

### By >= 2020

In [ ]:
trend6 = all_data.filter((f.year('date_release') >= lit('2020'))).groupBy("title",'date_release').count().orderBy(f.col('count').desc())

In [ ]:
trend6.show(10)